# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [ ]:
# Your code here

All good!


# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [ ]:
# Your code here

['catalogue/category/books/travel_2/index.html', 'catalogue/category/books/mystery_3/index.html', 'catalogue/category/books/historical-fiction_4/index.html', 'catalogue/category/books/sequential-art_5/index.html', 'catalogue/category/books/classics_6/index.html', 'catalogue/category/books/philosophy_7/index.html', 'catalogue/category/books/romance_8/index.html', 'catalogue/category/books/womens-fiction_9/index.html', 'catalogue/category/books/fiction_10/index.html', 'catalogue/category/books/childrens_11/index.html', 'catalogue/category/books/religion_12/index.html', 'catalogue/category/books/nonfiction_13/index.html', 'catalogue/category/books/music_14/index.html', 'catalogue/category/books/default_15/index.html', 'catalogue/category/books/science-fiction_16/index.html', 'catalogue/category/books/sports-and-games_17/index.html', 'catalogue/category/books/add-a-comment_18/index.html', 'catalogue/category/books/fantasy_19/index.html', 'catalogue/category/books/new-adult_20/index.html', 

# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [ ]:
# Your code here

# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [ ]:
# Your code here

# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [ ]:
from tqdm import tqdm

# Your code here

  0%|          | 0/50 [00:00<?, ?it/s]

Exploring category: travel_2





  2%|▏         | 1/50 [00:09<07:47,  9.54s/it]





Exploring category: mystery_3





  4%|▍         | 2/50 [00:26<10:57, 13.70s/it]

...





Exploring category: erotica_50





 98%|█████████▊| 49/50 [07:27<00:02,  2.23s/it]





Exploring category: crime_51





100%|██████████| 50/50 [07:29<00:00,  8.99s/it]

,Title,Price,Availability,Rating,Description,UPC,Category
0,It's Only the Himalayas,£45.17,In stock (19 available),Two,"“Wherever you go, whatever you do, just . . . ...",a22124811bfa8350,travel_2
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,In stock (15 available),Four,Acclaimed travel writer Rick Antonson sets his...,ce60436f52c5ee68,travel_2
2,See America: A Celebration of Our National Par...,£48.87,In stock (14 available),Three,To coincide with the 2016 centennial anniversa...,f9705c362f070608,travel_2
3,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,In stock (8 available),Two,With a new foreword by Tim Ferriss •There’s no...,1809259a5a5f1d8d,travel_2
4,Under the Tuscan Sun,£37.33,In stock (7 available),Three,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,a94350ee74deaa07,travel_2
